In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from typing import List, Tuple, Union
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from branca.element import Figure
from IPython.display import display
from tqdm.notebook import tqdm
import re
from functools import partial
import requests
from time import sleep

pd.set_option("display.max_rows", 20)

In [11]:
comm_pop_df = pd.read_csv("../data/processed/community_population_df.csv")

avg_occupancy_rate_df = pd.read_csv("../output/241110_131604/avg_occupancy_rate_each_hpid.csv")
er_df = pd.read_csv("../data/processed/emergency_df.csv")
avg_occupancy_rate_df = avg_occupancy_rate_df.merge(er_df, on="hpid", how="left")
avg_occupancy_rate_df = avg_occupancy_rate_df[
    ["hpid", "기관명", "위도", "경도", "의료기관분류", "hperyn", "avg_occupancy_rate"]
]

def preprocessing():
    # 응급실 병상 수 전처리
    avg_hperyn = avg_occupancy_rate_df.groupby("의료기관분류").hperyn.mean().round()
    for typ, avg in avg_hperyn.items():
        avg_occupancy_rate_df.loc[avg_occupancy_rate_df["의료기관분류"] == typ, "hperyn"] = \
            avg_occupancy_rate_df.loc[avg_occupancy_rate_df["의료기관분류"] == typ, "hperyn"].fillna(avg).replace(0, avg)
preprocessing()


In [12]:
def create_white_background_map(center_loc: List[float] = [36, 127.7], zoom_start: int = 6):
    f_map = folium.Map(location=center_loc, tiles="cartodbpositron", zoom_start=zoom_start)
    fig = Figure(width=500, height=400)
    fig.add_child(f_map)
    return fig, f_map

def add_residential_layer(f_map, res_df, radius_weight=0.03, color: str = "green"):
    for i in range(res_df.shape[0]):
        radius = np.sqrt(res_df["인구수"].iloc[i]) * radius_weight
        folium.CircleMarker(
            [res_df["위도"].iloc[i], res_df["경도"].iloc[i]], 
            radius=radius, color=color, stroke=False, fill=True, fill_opacity=0.5
        ).add_to(f_map)

def add_emergency_layer(f_map, er_df, radius_weight=0.1, color: str = "red"):
    for i in range(er_df.shape[0]):
        radius = np.sqrt(er_df["hperyn"].map(lambda x: int(x)).iloc[i]) * radius_weight
        opacity = 0.2 + er_df["avg_occupancy_rate"].iloc[i] / 100 * 0.8
        folium.CircleMarker(
            [er_df["위도"].iloc[i], er_df["경도"].iloc[i]], 
            radius=radius, color=color, stroke=False, fill=True, fill_opacity=opacity
        ).add_to(f_map)


In [13]:
tk = ["대구", "경북"]

fig, f_map = create_white_background_map(center_loc=[36.3, 128.7], zoom_start=8)
add_residential_layer(f_map, res_df=comm_pop_df[comm_pop_df.시도.isin(["대구", "경북"])])
add_emergency_layer(f_map, er_df=avg_occupancy_rate_df, radius_weight=3)
display(fig)

In [14]:
avg_occupancy_rate_df.sort_values("avg_occupancy_rate")

,hpid,기관명,위도,경도,의료기관분류,hperyn,avg_occupancy_rate
44,A2700070,의료법인영암의료재단고령영생병원,35.729955,128.272011,지역응급의료기관,8.0,7.604167
48,A2700071,봉화해성병원,36.891707,128.733501,응급의료기관 외의 의료기관(응급의료시설),10.0,8.166667
30,A2700011,상주적십자병원,36.414641,128.155601,지역응급의료기관,10.0,10.833333
28,A1300034,드림종합병원,35.839635,128.574355,지역응급의료기관,10.0,11.416667
33,A2702752,영주적십자병원,36.825258,128.604003,지역응급의료기관,10.0,11.666667
...,...,...,...,...,...,...,...
0,A1300002,경북대학교병원,35.866235,128.604315,권역응급의료센터,22.0,55.833333
24,A1300005,대구가톨릭대학교칠곡가톨릭병원,35.934880,128.549252,지역응급의료기관,10.0,56.166667
8,A1300009,대구파티마병원,35.883934,128.623835,지역응급의료센터,28.0,57.559524
52,A1300033,의료법인백천의료재단바로본병원,35.868455,128.615184,응급의료기관 외의 의료기관(응급의료시설),8.0,60.312500
